In [1]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Reading Session keys

In [2]:
session_key_list = pd.read_csv('session_keys.csv')

In [3]:
for i in session_key_list['session_key']:
    response = urlopen(f'https://api.openf1.org/v1/pit?session_key={i}')
    data = json.loads(response.read().decode('utf-8'))
    if i == 7953:
        df_pit = pd.DataFrame(data)
    else:
        hold = pd.DataFrame(data)
        df_pit = pd.concat([df_pit,hold])
        time.sleep(0.5)
    print(f'Session {i} extracted.')
print('Done')


Session 7953 extracted.
Session 7779 extracted.
Session 7787 extracted.
Session 9070 extracted.
Session 9078 extracted.
Session 9094 extracted.
Session 9102 extracted.
Session 9110 extracted.
Session 9118 extracted.
Session 9126 extracted.
Session 9133 extracted.
Session 9141 extracted.
Session 9149 extracted.
Session 9157 extracted.
Session 9165 extracted.
Session 9173 extracted.
Session 9221 extracted.
Session 9213 extracted.
Session 9181 extracted.
Session 9205 extracted.
Session 9189 extracted.
Session 9197 extracted.
Session 9472 extracted.
Session 9480 extracted.
Session 9488 extracted.
Session 9496 extracted.
Session 9673 extracted.
Session 9507 extracted.
Session 9515 extracted.
Session 9523 extracted.
Session 9531 extracted.
Session 9539 extracted.
Session 9550 extracted.
Session 9558 extracted.
Session 9566 extracted.
Session 9574 extracted.
Session 9582 extracted.
Session 9590 extracted.
Session 9598 extracted.
Session 9606 extracted.
Session 9617 extracted.
Session 9625 ext

In [4]:
df_pit

,date,session_key,meeting_key,driver_number,pit_duration,lap_number
0,2023-06-04T13:05:22.607000+00:00,9102,1211,4,37.7,1.0
1,2023-06-04T13:10:42.773000+00:00,9102,1211,77,23.6,5.0
2,2023-06-04T13:14:44.785000+00:00,9102,1211,27,22.2,8.0
3,2023-06-04T13:16:02.690000+00:00,9102,1211,24,23.4,9.0
4,2023-06-04T13:16:07.674000+00:00,9102,1211,21,22.5,9.0
...,...,...,...,...,...,...
18,2025-11-23T04:50:08.739000+00:00,9858,1274,22,20.6,27.0
19,2025-11-23T04:51:50.715000+00:00,9858,1274,30,21.2,28.0
20,2025-11-23T04:52:53.254000+00:00,9858,1274,44,21.6,29.0
21,2025-11-23T04:54:29.901000+00:00,9858,1274,27,22.3,30.0


In [7]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [8]:
df_pit.columns

Index(['date', 'session_key', 'meeting_key', 'driver_number', 'pit_duration',
       'lap_number'],
      dtype='object')

In [9]:
dtype_dict = {
    'date': types.DateTime,
    'session_key': types.String,
    'meeting_key': types.String,
    'driver_number': types.Integer,
    'pit_duration': types.Float,
    'lap_number': types.Integer
}

In [10]:
df_pit.to_sql(name = 'raw_pit', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

252